<a href="https://colab.research.google.com/github/harnalashok/deeplearning-sequences/blob/main/Glove_A_Closer_Look_at_Word_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# B - A Closer Look at Word Embeddings

We have very briefly covered how word embeddings (also known as word vectors) are used in the tutorials. In this appendix we'll have a closer look at these embeddings and find some (hopefully) interesting results.

Embeddings transform a one-hot encoded vector (a vector that is 0 in elements except one, which is 1) into a much smaller dimension vector of real numbers. The one-hot encoded vector is also known as a *sparse vector*, whilst the real valued vector is known as a *dense vector*. 

The key concept in these word embeddings is that words that appear in similar _contexts_ appear nearby in the vector space, i.e. the Euclidean distance between these two word vectors is small. By context here, we mean the surrounding words. For example in the sentences "I purchased some items at the shop" and "I purchased some items at the store" the words 'shop' and 'store' appear in the same context and thus should be close together in vector space.

You may have also heard about *word2vec*. *word2vec* is an algorithm (actually a bunch of algorithms) that calculates word vectors from a corpus. In this appendix we use *GloVe* vectors, *GloVe* being another algorithm to calculate word vectors. If you want to know how *word2vec* works, check out a two part series [here](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/) and [here](http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/), and if you want to find out more about *GloVe*, check the website [here](https://nlp.stanford.edu/projects/glove/).

In PyTorch, we use word vectors with the `nn.Embedding` layer, which takes a _**[sentence length, batch size]**_ tensor and transforms it into a _**[sentence length, batch size, embedding dimensions]**_ tensor.

In tutorial 2 onwards, we also used pre-trained word embeddings (specifically the GloVe vectors) provided by TorchText. These embeddings have been trained on a gigantic corpus. We can use these pre-trained vectors within any of our models, with the idea that as they have already learned the context of each word they will give us a better starting point for our word vectors. This usually leads to faster training time and/or improved accuracy.

In this appendix we won't be training any models, instead we'll be looking at the word embeddings and finding a few interesting things about them.

A lot of the code from the first half of this appendix is taken from [here](https://github.com/spro/practical-pytorch/blob/master/glove-word-vectors/glove-word-vectors.ipynb). For more information about word embeddings, go [here](https://monkeylearn.com/blog/word-embeddings-transform-text-numbers/). 

## Loading the GloVe vectors

First, we'll load the GloVe vectors. The `name` field specifies what the vectors have been trained on, here the `6B` means a corpus of 6 billion words. The `dim` argument specifies the dimensionality of the word vectors. GloVe vectors are available in 50, 100, 200 and 300 dimensions. There is also a `42B` and `840B` glove vectors, however they are only available at 300 dimensions.

In [ ]:
import torchtext.vocab

glove = torchtext.vocab.GloVe(name = '6B', dim = 100)

print(f'There are {len(glove.itos)} words in the vocabulary')

There are 400000 words in the vocabulary


As shown above, there are 400,000 unique words in the GloVe vocabulary. These are the most common words found in the corpus the vectors were trained on. **In these set of GloVe vectors, every single word is lower-case only.**

`glove.vectors` is the actual tensor containing the values of the embeddings.

In [ ]:
glove.vectors.shape

torch.Size([400000, 100])

We can see what word is associated with each row by checking the `itos` (int to string) list. 

Below implies that row 0 is the vector associated with the word 'the', row 1 for ',' (comma), row 2 for '.' (period), etc.

In [ ]:
glove.itos[:10]

['the', ',', '.', 'of', 'to', 'and', 'in', 'a', '"', "'s"]

We can also use the `stoi` (string to int) dictionary, in which we input a word and receive the associated integer/index. If you try get the index of a word that is not in the vocabulary, you receive an error.

In [ ]:
glove.stoi['the']

0

We can get the vector of a word by first getting the integer associated with it and then indexing into the word embedding tensor with that index.

In [ ]:
glove.vectors[glove.stoi['the']].shape

torch.Size([100])

We'll be doing this a lot, so we'll create a function that takes in word embeddings and a word then returns the associated vector. It'll also throw an error if the word doesn't exist in the vocabulary.

In [ ]:
def get_vector(embeddings, word):
    assert word in embeddings.stoi, f'*{word}* is not in the vocab!'
    return embeddings.vectors[embeddings.stoi[word]]

As before, we use a word to get the associated vector.

In [ ]:
get_vector(glove, 'the').shape

torch.Size([100])

## Similar Contexts

Now to start looking at the context of different words. 

If we want to find the words similar to a certain input word, we first find the vector of this input word, then we scan through our vocabulary calculating the distance between the vector of each word and our input word vector. We then sort these from closest to furthest away.

The function below returns the closest 10 words to an input word vector:

In [ ]:
import torch

def closest_words(embeddings, vector, n = 10):
    
    distances = [(word, torch.dist(vector, get_vector(embeddings, word)).item())
                 for word in embeddings.itos]
    
    return sorted(distances, key = lambda w: w[1])[:n]

Let's try it out with 'korea'. The closest word is the word 'korea' itself (not very interesting), however all of the words are related in some way. Pyongyang is the capital of North Korea, DPRK is the official name of North Korea, etc.

Interestingly, we also get 'Japan' and 'China',  implies that Korea, Japan and China are frequently talked about together in similar contexts. This makes sense as they are geographically situated near each other. 

In [ ]:
word_vector = get_vector(glove, 'korea')

closest_words(glove, word_vector)

[('korea', 0.0),
 ('pyongyang', 3.9039554595947266),
 ('korean', 4.068886756896973),
 ('dprk', 4.2631049156188965),
 ('seoul', 4.340494155883789),
 ('japan', 4.551243782043457),
 ('koreans', 4.615609169006348),
 ('south', 4.65822696685791),
 ('china', 4.839518070220947),
 ('north', 4.986356735229492)]

Looking at another country, India, we also get nearby countries: Thailand, Malaysia and Sri Lanka (as two separate words). Australia is relatively close to India (geographically), but Thailand and Malaysia are closer. So why is Australia closer to India in vector space? This is most probably due to India and Australia appearing in the context of [cricket](https://en.wikipedia.org/wiki/Cricket) matches together.

In [ ]:
word_vector = get_vector(glove, 'india')

closest_words(glove, word_vector)

[('india', 0.0),
 ('pakistan', 3.6954822540283203),
 ('indian', 4.114313125610352),
 ('delhi', 4.155975818634033),
 ('bangladesh', 4.261017799377441),
 ('lanka', 4.435845851898193),
 ('sri', 4.515716552734375),
 ('australia', 4.806082725524902),
 ('thailand', 4.994781017303467),
 ('malaysia', 5.009334087371826)]

We'll also create another function that will nicely print out the tuples returned by our `closest_words` function.

In [ ]:
def print_tuples(tuples):
    for w, d in tuples:
        print(f'({d:02.04f}) {w}') 

A final word to look at, 'sports'. As we can see, the closest words are most of the sports themselves. 

In [ ]:
word_vector = get_vector(glove, 'sports')

print_tuples(closest_words(glove, word_vector))

(0.0000) sports
(3.5875) sport
(4.4590) soccer
(4.6508) basketball
(4.6561) baseball
(4.8028) sporting
(4.8763) football
(4.9624) professional
(4.9824) entertainment
(5.0975) media


## Analogies

Another property of word embeddings is that they can be operated on just as any standard vector and give interesting results.

We'll show an example of this first, and then explain it:

In [ ]:
def analogy(embeddings, word1, word2, word3, n=5):
    
    #get vectors for each word
    word1_vector = get_vector(embeddings, word1)
    word2_vector = get_vector(embeddings, word2)
    word3_vector = get_vector(embeddings, word3)
    
    #calculate analogy vector
    analogy_vector = word2_vector - word1_vector + word3_vector
    
    #find closest words to analogy vector
    candidate_words = closest_words(embeddings, analogy_vector, n+3)
    
    #filter out words already in analogy
    candidate_words = [(word, dist) for (word, dist) in candidate_words 
                       if word not in [word1, word2, word3]][:n]
    
    print(f'{word1} is to {word2} as {word3} is to...')
    
    return candidate_words

In [ ]:
print_tuples(analogy(glove, 'man', 'king', 'woman'))

man is to king as woman is to...
(4.0811) queen
(4.6429) monarch
(4.9055) throne
(4.9216) elizabeth
(4.9811) prince


This is the canonical example which shows off this property of word embeddings. So why does it work? Why does the vector of 'woman' added to the vector of 'king' minus the vector of 'man' give us 'queen'?

If we think about it, the vector calculated from 'king' minus 'man' gives us a "royalty vector". This is the vector associated with traveling from a man to his royal counterpart, a king. If we add this "royality vector" to 'woman', this should travel to her royal equivalent, which is a queen!

We can do this with other analogies too. For example, this gets an "acting career vector":

In [ ]:
print_tuples(analogy(glove, 'man', 'actor', 'woman'))

man is to actor as woman is to...
(2.8133) actress
(5.0039) comedian
(5.1399) actresses
(5.2773) starred
(5.3085) screenwriter


For a "baby animal vector":

In [ ]:
print_tuples(analogy(glove, 'cat', 'kitten', 'dog'))

cat is to kitten as dog is to...
(3.8146) puppy
(4.2944) rottweiler
(4.5888) puppies
(4.6086) pooch
(4.6520) pug


A "capital city vector":

In [ ]:
print_tuples(analogy(glove, 'france', 'paris', 'england'))

france is to paris as england is to...
(4.1426) london
(4.4938) melbourne
(4.7087) sydney
(4.7630) perth
(4.7952) birmingham


A "musician's genre vector":

In [ ]:
print_tuples(analogy(glove, 'elvis', 'rock', 'eminem'))

elvis is to rock as eminem is to...
(5.6597) rap
(6.2057) rappers
(6.2161) rapper
(6.2444) punk
(6.2690) hop


And an "ingredient vector":

In [ ]:
print_tuples(analogy(glove, 'beer', 'barley', 'wine'))

beer is to barley as wine is to...
(5.6021) grape
(5.6760) beans
(5.8174) grapes
(5.9035) lentils
(5.9454) figs


## Correcting Spelling Mistakes

Another interesting property of word embeddings is that they can actually be used to correct spelling mistakes! 

We'll put their findings into code and briefly explain them, but to read more about this, check out the [original thread](http://forums.fast.ai/t/nlp-any-libraries-dictionaries-out-there-for-fixing-common-spelling-errors/16411) and the associated [write-up](https://blog.usejournal.com/a-simple-spell-checker-built-from-word-vectors-9f28452b6f26).

First, we need to load up the much larger vocabulary GloVe vectors, this is due to the spelling mistakes not appearing in the smaller vocabulary. 

**Note**: these vectors are very large (~2GB), so watch out if you have a limited internet connection.

In [ ]:
glove = torchtext.vocab.GloVe(name = '840B', dim = 300)

Checking the vocabulary size of these embeddings, we can see we now have over 2 million unique words in our vocabulary!

In [ ]:
glove.vectors.shape

torch.Size([2196017, 300])

As the vectors were trained with a much larger vocabulary on a larger corpus of text, the words that appear are a little different. Notice how the words 'north', 'south', 'pyongyang' and 'dprk' no longer appear in the most closest words to 'korea'.

In [ ]:
word_vector = get_vector(glove, 'korea')

print_tuples(closest_words(glove, word_vector))

(0.0000) korea
(3.9857) taiwan
(4.4022) korean
(4.9016) asia
(4.9593) japan
(5.0721) seoul
(5.4058) thailand
(5.6025) singapore
(5.7010) russia
(5.7240) hong


Our first step to correcting spelling mistakes is looking at the vector for a misspelling of the word 'reliable'.

In [ ]:
word_vector = get_vector(glove, 'relieable')

print_tuples(closest_words(glove, word_vector))

(0.0000) relieable
(5.0366) relyable
(5.2610) realible
(5.4719) realiable
(5.5402) relable
(5.5917) relaible
(5.6412) reliabe
(5.8802) relaiable
(5.9593) stabel
(5.9981) consitant


Notice how the correct spelling, "reliable", does not appear in the top 10 closest words. Surely the misspellings of a word should appear next to the correct spelling of the word as they appear in the same context, right? 

The hypothesis is that misspellings of words are all equally shifted away from their correct spelling. This is because articles of text that contain spelling mistakes are usually written in an informal manner where correct spelling doesn't matter as much (such as tweets/blog posts), thus spelling errors will appear together as they appear in context of informal articles.

Similar to how we created analogies before, we can create a "correct spelling" vector. This time, instead of using a single example to create our vector, we'll use the average of multiple examples. This will hopefully give better accuracy!

We first create a vector for the correct spelling, 'reliable', then calculate the difference between the "reliable vector" and each of the 8 misspellings of 'reliable'. As we are going to concatenate these 8 misspelling tensors together we need to unsqueeze a "batch" dimension to them.

In [ ]:
reliable_vector = get_vector(glove, 'reliable')

reliable_misspellings = ['relieable', 'relyable', 'realible', 'realiable', 
                         'relable', 'relaible', 'reliabe', 'relaiable']

diff_reliable = [(reliable_vector - get_vector(glove, s)).unsqueeze(0) 
                 for s in reliable_misspellings]

We take the average of these 8 'difference from reliable' vectors to get our "misspelling vector".

In [ ]:
misspelling_vector = torch.cat(diff_reliable, dim = 0).mean(dim = 0)

We can now correct other spelling mistakes using this "misspelling vector" by finding the closest words to the sum of the vector of a misspelled word and the "misspelling vector".

For a misspelling of "because":

In [ ]:
word_vector = get_vector(glove, 'becuase')

print_tuples(closest_words(glove, word_vector + misspelling_vector))

(6.1090) because
(6.4250) even
(6.4358) fact
(6.4914) sure
(6.5094) though
(6.5601) obviously
(6.5682) reason
(6.5856) if
(6.6099) but
(6.6415) why


For a misspelling of "definitely":

In [ ]:
word_vector = get_vector(glove, 'defintiely')

print_tuples(closest_words(glove, word_vector + misspelling_vector))

(5.4070) definitely
(5.5643) certainly
(5.7192) sure
(5.8152) well
(5.8588) always
(5.8812) also
(5.9557) simply
(5.9667) consider
(5.9821) probably
(5.9948) definately


For a misspelling of "consistent":

In [ ]:
word_vector = get_vector(glove, 'consistant')

print_tuples(closest_words(glove, word_vector + misspelling_vector))

(5.9641) consistent
(6.3674) reliable
(7.0195) consistant
(7.0299) consistently
(7.1605) accurate
(7.2737) fairly
(7.3037) good
(7.3520) reasonable
(7.3801) dependable
(7.4027) ensure


For a misspelling of "package":

In [ ]:
word_vector = get_vector(glove, 'pakage')

print_tuples(closest_words(glove, word_vector + misspelling_vector))

(6.6117) package
(6.9315) packages
(7.0195) pakage
(7.0911) comes
(7.1241) provide
(7.1469) offer
(7.1861) reliable
(7.2431) well
(7.2434) choice
(7.2453) offering


For a more in-depth look at this, check out the [write-up](https://blog.usejournal.com/a-simple-spell-checker-built-from-word-vectors-9f28452b6f26).